In [1]:
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir("/content/drive/MyDrive/ResearchProject")

Mounted at /content/drive/


In [2]:
from irp_functions import *

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random

from scipy.stats import ks_2samp
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, confusion_matrix, f1_score 
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score, precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.base import clone
from sklearn.neighbors import NearestNeighbors

In [3]:
root_path = '/content/drive/My Drive/ResearchProject/' 
dataset_path = root_path + 'Datasets/'
wine_filename = root_path + 'Datasets/wine_quality.csv'
df = pd.read_csv(wine_filename)
df['Minority'] = np.where(df['quality'] == 8, 1, 0)
df.drop('quality', axis=1, inplace=True)
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,Minority
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,0
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,0
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,0
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,0
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,0
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,0
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,0
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,0
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,0


In [4]:
y = df['Minority'].to_numpy()
X = df.drop(['Minority'], axis=1).to_numpy()

In [5]:
benchmark = ten_experiments(X=X, y=y)

In [6]:
benchmark

{1: [{'ROCAUC': 0.9615097624673549, 'confusion_matrix': array([[473,   0],
          [  9,   8]]), 'f1': 0.6399999999999999, 'precision': 1.0, 'recall': 0.47058823529411764},
  {'ROCAUC': 0.8898146996642209, 'confusion_matrix': array([[473,   0],
          [  6,  11]]), 'f1': 0.7857142857142858, 'precision': 1.0, 'recall': 0.6470588235294118},
  {'ROCAUC': 0.9168013928615844, 'confusion_matrix': array([[473,   0],
          [ 11,   6]]), 'f1': 0.5217391304347826, 'precision': 1.0, 'recall': 0.35294117647058826},
  {'ROCAUC': 0.8483992467043313, 'confusion_matrix': array([[469,   3],
          [ 12,   6]]), 'f1': 0.4444444444444444, 'precision': 0.6666666666666666, 'recall': 0.3333333333333333},
  {'ROCAUC': 0.8888888888888888, 'confusion_matrix': array([[471,   1],
          [ 11,   7]]), 'f1': 0.5384615384615385, 'precision': 0.875, 'recall': 0.3888888888888889},
  {'ROCAUC': 0.9404425612052731, 'confusion_matrix': array([[472,   0],
          [  5,  13]]), 'f1': 0.8387096774193548, '

In [7]:
benchmark_data = get_DataFrame_from_results(benchmark)
benchmark_data

,TN,FN,TP,FP,ROCAUC,f1,precision,recall
0,473.0,9.0,8.0,0.0,0.961510,0.640000,1.000000,0.470588
1,473.0,6.0,11.0,0.0,0.889815,0.785714,1.000000,0.647059
2,473.0,11.0,6.0,0.0,0.916801,0.521739,1.000000,0.352941
3,469.0,12.0,6.0,3.0,0.848399,0.444444,0.666667,0.333333
4,471.0,11.0,7.0,1.0,0.888889,0.538462,0.875000,0.388889
...,...,...,...,...,...,...,...,...
95,472.0,9.0,9.0,0.0,0.889124,0.666667,1.000000,0.500000
96,471.0,10.0,8.0,1.0,0.902954,0.592593,0.888889,0.444444
97,469.0,7.0,11.0,3.0,0.956803,0.687500,0.785714,0.611111
98,471.0,9.0,8.0,1.0,0.924227,0.615385,0.888889,0.470588


In [8]:
SMOTE = {}
for oversampling_rate in range(100, 1100, 100):
  results = ten_experiments(X=X, y=y, over_sampler="SMOTE", N=oversampling_rate)
  data = get_DataFrame_from_results(results)
  SMOTE[oversampling_rate] = data

In [9]:
Outlier_SMOTE_Euclidean = {}
for oversampling_rate in range(100, 1100, 100):
  results = ten_experiments(X=X, y=y, over_sampler="Outlier_SMOTE", 
                            N=oversampling_rate)
  data = get_DataFrame_from_results(results)
  Outlier_SMOTE_Euclidean[oversampling_rate] = data

In [10]:
Outlier_SMOTE_Manhattan = {}
for oversampling_rate in range(100, 1100, 100):
  results = ten_experiments(X=X, y=y, over_sampler="Outlier_SMOTE", 
                            metric="manhattan" ,N=oversampling_rate)
  data = get_DataFrame_from_results(results)
  Outlier_SMOTE_Manhattan[oversampling_rate] = data

In [11]:
Outlier_SMOTE_Chebyshev = {}
for oversampling_rate in range(100, 1100, 100):
  results = ten_experiments(X=X, y=y, over_sampler="Outlier_SMOTE", 
                            metric="chebyshev" ,N=oversampling_rate)
  data = get_DataFrame_from_results(results)
  Outlier_SMOTE_Chebyshev[oversampling_rate] = data

In [12]:
os.chdir("/content/drive/MyDrive/ResearchProject/Results/Wine")

In [13]:
benchmark_data.to_csv('benchmark.csv')

In [14]:
for oversampling_rate in range(100, 1100, 100):
  prefix = str(oversampling_rate)
  
  smote = SMOTE[oversampling_rate]
  euclidean = Outlier_SMOTE_Euclidean[oversampling_rate]
  manhattan = Outlier_SMOTE_Manhattan[oversampling_rate]
  chebyshev = Outlier_SMOTE_Chebyshev[oversampling_rate]

  smote_name = prefix + "_SMOTE.csv"
  euclidean_name = prefix + "_Euclidean_Outlier_SMOTE.csv"
  manhattan_name = prefix + "_Manhattan_Outlier_SMOTE.csv"
  chebyshev_name = prefix + "_Chebyshev_Outlier_SMOTE.csv"

  smote.to_csv(smote_name)
  euclidean.to_csv(euclidean_name)
  manhattan.to_csv(manhattan_name)
  chebyshev.to_csv(chebyshev_name)  